In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import os
os.chdir("/content/drive/MyDrive/ML project 2/")
os.getcwd()

'/content/drive/MyDrive/ML project 2'

In [ ]:
rm -rf train_data

In [3]:
ls

'Crack detection'/          ResNet50.ipynb
 data_augmentation.ipynb   'ResNet50 results.txt'
 EfficientNetB1.ipynb       test_data_1outOf10_fps/
 EfficientNetB7.ipynb       train_data1_rep.zip
'first two runs ENB1.jpg'  'use colab.ipynb'
 framing_video.py           vgg16_training.ipynb
 getframes.py               Videos/
'last two runs ENB1.jpg'   'videos inspections'/
 Literature/                Videos_og.lnk@
 load_model.ipynb          'Zoomed and Paused videos'/
 ML_experiment.ipynb


In [4]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [5]:
import cv2 as cv
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, Dropout, Input, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D, AveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

In [ ]:
!unzip "train_data1_rep"

Streaminguitvoer ingekort tot de laatste 5000 regels.
  inflating: train_data/train/Fissure/FissureTransH522.jpg  
  inflating: train_data/train/Fissure/FissureTransH523.jpg  
  inflating: train_data/train/Fissure/FissureTransH524.jpg  
  inflating: train_data/train/Fissure/FissureTransH525.jpg  
  inflating: train_data/train/Fissure/FissureTransH526.jpg  
  inflating: train_data/train/Fissure/FissureTransH527.jpg  
  inflating: train_data/train/Fissure/FissureTransH528.jpg  
  inflating: train_data/train/Fissure/FissureTransH529.jpg  
  inflating: train_data/train/Fissure/FissureTransH53.jpg  
  inflating: train_data/train/Fissure/FissureTransH530.jpg  
  inflating: train_data/train/Fissure/FissureTransH531.jpg  
  inflating: train_data/train/Fissure/FissureTransH532.jpg  
  inflating: train_data/train/Fissure/FissureTransH533.jpg  
  inflating: train_data/train/Fissure/FissureTransH534.jpg  
  inflating: train_data/train/Fissure/FissureTransH535.jpg  
  inflating: train_data/train/Fi

In [ ]:
train_path = "train_data/train"
valid_path = "train_data/valid"
test_path = "train_data/test"

In [ ]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.efficientnet.preprocess_input) \
    .flow_from_directory(directory=train_path, target_size=(224,224), classes=['Fissure', 'Racines', 'Normal'], batch_size=10)

valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.efficientnet.preprocess_input) \
    .flow_from_directory(directory=valid_path, target_size=(224,224), classes=['Fissure', 'Racines', 'Normal'], batch_size=10)

test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.efficientnet.preprocess_input) \
    .flow_from_directory(directory=test_path, target_size=(224,224), classes=['Fissure', 'Racines', 'Normal'], batch_size=10, shuffle=False)

Found 68283 images belonging to 3 classes.
Found 700 images belonging to 3 classes.
Found 600 images belonging to 3 classes.


In [ ]:
efficientnet_model = tf.keras.applications.efficientnet.EfficientNetB7(include_top=False,input_tensor=Input(shape=(224, 224, 3)))

258088960/258076736 [==============================] - 4s 0us/step


In [ ]:
efficientnet_model.summary()

In [ ]:
top_layers = efficientnet_model.output
top_layers = Flatten(name="top_flattening")(top_layers)
top_layers = Dense(1024, activation="ReLU",name="first_dense_top")(top_layers)
top_layers = Dropout(0.5,name="top_dropout")(top_layers)
top_layers = Dense(units=3, activation="softmax",name="linear_output")(top_layers)

In [ ]:
model = Model(inputs=efficientnet_model.input, outputs=top_layers)

In [ ]:
for layer in efficientnet_model.layers:
	layer.trainable = False

In [ ]:
learning_rate = np.logspace(-4,-2,4)
learning_rate = learning_rate[1:]
print(learning_rate)

[0.00046416 0.00215443 0.01      ]


In [ ]:
for lr in learning_rate:
  print("--------------------")
  print("learning rate = ",lr)
  print("--------------------")
  model.compile(optimizer=Adam(learning_rate=lr), loss='categorical_crossentropy', metrics=['accuracy'])
  model.fit(train_batches,
            steps_per_epoch=len(train_batches),
            validation_data=valid_batches,
            validation_steps=len(valid_batches),
            epochs=5
  )

--------------------
learning rate =  0.00046415888336127773
--------------------
Epoch 1/5
  19/6829 [..............................] - ETA: 9:46:26 - loss: 9.7564 - accuracy: 0.7211

KeyboardInterrupt: ignored

In [ ]:
predictions = model.predict(x=test_batches, steps=len(test_batches), verbose=0)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
import itertools

In [ ]:
cm = confusion_matrix(y_true=test_batches.classes, y_pred=np.argmax(predictions, axis=-1))
cm_plot_labels = ['Fissure','Racines', 'Normal']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')